In [24]:
#pip install category-encoders==2.5.0

In [25]:
#pip install nbformat==5.1.3

In [26]:
#pip install mlxtend==0.20.0

In [27]:
import pandas as pd
import numpy as np
#https://stackabuse.com/implementing-lda-in-python-with-scikit-learn/

############################ For regression: f_regression, mutual_info_regression
############################ For classification: chi2, f_classif, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, BaggingRegressor, AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.decomposition import PCA, LatentDirichletAllocation
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import r2_score, confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE, ADASYN
%matplotlib inline
from sklearn.linear_model import RidgeCV
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression, f_classif, mutual_info_classif, mutual_info_regression
from time import time
from matplotlib.colors import ListedColormap
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
#import category_encoders as ce
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#https://jovian.ai/iguedeominiabohs/profit-prediction-of-1000-companies
#from mlxtend.evaluate import bias_variance_decomp
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso, LassoCV

In [28]:
# from google.colab import drive
# drive.mount('/content/drive')

In [29]:
# IMPORTS FOR BOSSTING AND BAGGING CLASSIFIERS
# BAGGING = BOOTSTRAP  + AGGREGATION  = USED FOR HIGH VARIANCE
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


#BOOSTING =  USED FOR HIGH BIAS
# ADABOOST CLASSIFIER
from sklearn.ensemble import AdaBoostClassifier
# STOCHASTIC GRADIENT CLASSIFIER
from sklearn.ensemble import GradientBoostingClassifier


# VOTING CLASSIFIER
from sklearn.ensemble import VotingClassifier
# LINEAR AND SVC TO BE USED  INSIDE THE VOTING CLASSIFIER
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [30]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/cpmp.csv")
df.head(5)


,instance_id,repetition,stacks,tiers,stack.tier.ratio,container.density,empty.stack.pct,overstowing.stack.pct,overstowing.2cont.stack.pct,group.same.min,...,overstowage.pct,bflb,left.density,tier.weighted.groups,avg.l1.top.left.lg.group,cont.empty.grt.estack,pct.bottom.pct.on.top,runtime,runstatus,algorithm
0,BF1_cpmp_16_5_48_10_29_1,1,16,5,0.3125,0.6,0.0625,0.6250,0.909091,0,...,0.3625,29,0.382222,0.441162,0.509615,0.1750,0.0,0.004,ok,idastar.symmulgt.transmul
1,BF1_cpmp_16_5_48_10_29_12,1,16,5,0.3125,0.6,0.1875,0.5625,1.000000,0,...,0.3625,29,0.413333,0.427399,0.663462,0.3375,0.0,0.572,ok,astar.symmullt.transmul
2,BF1_cpmp_16_5_48_10_29_14,1,16,5,0.3125,0.6,0.1875,0.6250,0.833333,0,...,0.3625,29,0.111111,0.405997,0.615385,0.2625,0.0,0.764,ok,idastar.symmulgt.transmul
3,BF1_cpmp_16_5_48_10_29_15,1,16,5,0.3125,0.6,0.0625,0.5000,0.888889,0,...,0.3625,29,0.677778,0.409975,0.576923,0.1125,0.0,106.755,ok,idastar.symmulgt.transmul
4,BF1_cpmp_16_5_48_10_29_17,1,16,5,0.3125,0.6,0.1250,0.5625,0.900000,0,...,0.3625,29,0.471111,0.428914,0.569231,0.3375,0.0,0.008,ok,idastar.symmulgt.transmul


In [31]:
df.columns

Index(['instance_id', 'repetition', 'stacks', 'tiers', 'stack.tier.ratio',
       'container.density', 'empty.stack.pct', 'overstowing.stack.pct',
       'overstowing.2cont.stack.pct', 'group.same.min', 'group.same.max',
       'group.same.mean', 'group.same.stdev', 'top.good.min', 'top.good.max',
       'top.good.mean', 'top.good.stdev', 'overstowage.pct', 'bflb',
       'left.density', 'tier.weighted.groups', 'avg.l1.top.left.lg.group',
       'cont.empty.grt.estack', 'pct.bottom.pct.on.top', 'runtime',
       'runstatus', 'algorithm'],
      dtype='object')

In [32]:
df = df[['stack.tier.ratio','container.density','empty.stack.pct','overstowing.stack.pct','overstowing.2cont.stack.pct','group.same.min','group.same.max','group.same.mean', 'group.same.stdev', 'top.good.min', 'top.good.max','top.good.mean', 'top.good.stdev', 'overstowage.pct', 'bflb','left.density', 'tier.weighted.groups', 'avg.l1.top.left.lg.group','cont.empty.grt.estack', 'runtime', 'algorithm']]

In [33]:
df.head(5)

,stack.tier.ratio,container.density,empty.stack.pct,overstowing.stack.pct,overstowing.2cont.stack.pct,group.same.min,group.same.max,group.same.mean,group.same.stdev,top.good.min,...,top.good.mean,top.good.stdev,overstowage.pct,bflb,left.density,tier.weighted.groups,avg.l1.top.left.lg.group,cont.empty.grt.estack,runtime,algorithm
0,0.3125,0.6,0.0625,0.6250,0.909091,0,6,4.36364,1.55346,2,...,4.60000,2.00998,0.3625,29,0.382222,0.441162,0.509615,0.1750,0.004,idastar.symmulgt.transmul
1,0.3125,0.6,0.1875,0.5625,1.000000,0,7,4.36364,1.61091,2,...,4.55556,2.26623,0.3625,29,0.413333,0.427399,0.663462,0.3375,0.572,astar.symmullt.transmul
2,0.3125,0.6,0.1875,0.6250,0.833333,0,7,4.36364,1.72008,2,...,4.30000,1.84662,0.3625,29,0.111111,0.405997,0.615385,0.2625,0.764,idastar.symmulgt.transmul
3,0.3125,0.6,0.0625,0.5000,0.888889,0,6,4.36364,1.61091,2,...,3.50000,2.29129,0.3625,29,0.677778,0.409975,0.576923,0.1125,106.755,idastar.symmulgt.transmul
4,0.3125,0.6,0.1250,0.5625,0.900000,0,6,4.36364,1.55346,2,...,4.66667,2.44949,0.3625,29,0.471111,0.428914,0.569231,0.3375,0.008,idastar.symmulgt.transmul


In [34]:
le = LabelEncoder()
df['algorithm'] = le.fit_transform(df['algorithm'])

In [35]:
df.head(5)

,stack.tier.ratio,container.density,empty.stack.pct,overstowing.stack.pct,overstowing.2cont.stack.pct,group.same.min,group.same.max,group.same.mean,group.same.stdev,top.good.min,...,top.good.mean,top.good.stdev,overstowage.pct,bflb,left.density,tier.weighted.groups,avg.l1.top.left.lg.group,cont.empty.grt.estack,runtime,algorithm
0,0.3125,0.6,0.0625,0.6250,0.909091,0,6,4.36364,1.55346,2,...,4.60000,2.00998,0.3625,29,0.382222,0.441162,0.509615,0.1750,0.004,2
1,0.3125,0.6,0.1875,0.5625,1.000000,0,7,4.36364,1.61091,2,...,4.55556,2.26623,0.3625,29,0.413333,0.427399,0.663462,0.3375,0.572,1
2,0.3125,0.6,0.1875,0.6250,0.833333,0,7,4.36364,1.72008,2,...,4.30000,1.84662,0.3625,29,0.111111,0.405997,0.615385,0.2625,0.764,2
3,0.3125,0.6,0.0625,0.5000,0.888889,0,6,4.36364,1.61091,2,...,3.50000,2.29129,0.3625,29,0.677778,0.409975,0.576923,0.1125,106.755,2
4,0.3125,0.6,0.1250,0.5625,0.900000,0,6,4.36364,1.55346,2,...,4.66667,2.44949,0.3625,29,0.471111,0.428914,0.569231,0.3375,0.008,2


In [36]:
X =  df.iloc[:,:20].values
X

array([[3.12500e-01, 6.00000e-01, 6.25000e-02, ..., 5.09615e-01,
        1.75000e-01, 4.00000e-03],
       [3.12500e-01, 6.00000e-01, 1.87500e-01, ..., 6.63462e-01,
        3.37500e-01, 5.72000e-01],
       [3.12500e-01, 6.00000e-01, 1.87500e-01, ..., 6.15385e-01,
        2.62500e-01, 7.64000e-01],
       ...,
       [5.00000e-01, 7.50000e-01, 0.00000e+00, ..., 5.52083e-01,
        0.00000e+00, 4.84000e-01],
       [5.00000e-01, 7.50000e-01, 8.33333e-02, ..., 4.37500e-01,
        1.38889e-01, 2.34330e+01],
       [5.00000e-01, 7.50000e-01, 0.00000e+00, ..., 5.76389e-01,
        0.00000e+00, 7.44530e+01]])

In [37]:
Y = df.iloc[:,20].values
Y

array([2, 1, 2, 2, 2, 0, 2, 2, 1, 1, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3,
       0, 3, 2, 2, 0, 1, 2, 0, 2, 0, 0, 0, 1, 1, 2, 0, 1, 0, 2, 0, 2, 0,
       2, 2, 2, 2, 2, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 0, 2, 2, 2,
       2, 2, 0, 2, 0, 0, 0, 0, 2, 1, 1, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2,
       2, 3, 2, 0, 0, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 3,
       0, 1, 1, 0, 1, 1, 1, 0, 3, 1, 0, 0, 0, 1, 0, 0, 1, 2, 2, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 2, 2, 0, 2, 2, 2, 2, 1, 1, 3, 0, 0, 3, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 3, 0, 2, 2, 1, 2, 2, 2, 1, 2, 2, 3, 1,
       3, 0, 0, 3, 3, 1, 3, 0, 1, 3, 3, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 3, 1, 2, 0, 3, 2, 2, 2, 2, 3, 3, 3, 1, 2, 3,
       0, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 3, 3, 1, 2, 2, 1, 2, 2, 0, 2, 3,
       3, 0, 3, 0, 2, 2, 2, 0, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3,
       0, 2, 2, 3, 3, 2, 1, 3, 3, 2, 3, 3, 3, 3, 3, 1, 3, 1, 3, 3, 1, 0,
       3, 3, 0, 2, 3, 0, 3, 2, 3, 2, 3, 1, 3, 2, 1,

In [38]:
X_train,X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [39]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)
print(type(X_train))
print(type(Y_train))

(421, 20)
(106, 20)
(421,)
(106,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [40]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


def evaluate(model, X_train, X_test, y_train, y_test):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    print("TRAINIG RESULTS: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_train, y_train_pred, output_dict=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_train, y_train_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_train, y_train_pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")

    print("TESTING RESULTS: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_test, y_test_pred, output_dict=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_test, y_test_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_test, y_test_pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")

In [41]:
# BAGGING CLASSIFIER MODEL WITH  ESTIMATOR AS DECISIONTREE WHICH IS ALSO A DEFAULT ESTIMATOR
tree = DecisionTreeClassifier()
bagging_clf = BaggingClassifier(base_estimator=tree, n_estimators=1500, random_state=42)
bagging_clf.fit(X_train, Y_train)

evaluate(bagging_clf, X_train, X_test, Y_train, Y_test)

# RESULTS SHOW OVER FITTING AS THE TRAIN ACCURACY WAS GOOD LEARNT I.E 100%
# TEST ACCURACY WAS BAD AS  THE CHANGED DATASET RESULTED IN LESS ACCURACY

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[ 58   0   0   0]
 [  0  63   0   0]
 [  0   0 128   0]
 [  0   0   0 172]]
ACCURACY SCORE:
1.0000
CLASSIFICATION REPORT:
              0     1      2      3  accuracy  macro avg  weighted avg
precision   1.0   1.0    1.0    1.0       1.0        1.0           1.0
recall      1.0   1.0    1.0    1.0       1.0        1.0           1.0
f1-score    1.0   1.0    1.0    1.0       1.0        1.0           1.0
support    58.0  63.0  128.0  172.0       1.0      421.0         421.0
TESTING RESULTS: 
CONFUSION MATRIX:
[[ 7  5  4  4]
 [ 2  4  7  8]
 [ 0  6 19  4]
 [ 2  6  2 26]]
ACCURACY SCORE:
0.5283
CLASSIFICATION REPORT:
                   0          1          2          3  accuracy   macro avg  \
precision   0.636364   0.190476   0.593750   0.619048  0.528302    0.509909   
recall      0.350000   0.190476   0.655172   0.722222  0.528302    0.479468   
f1-score    0.451613   0.190476   0.622951   0.666667  0.528302    0.482927   
support    20.000000  21.00

In [42]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=42, n_estimators=1000)
rf_clf.fit(X_train, Y_train)
evaluate(rf_clf, X_train, X_test, Y_train, Y_test)
# SAME RESULTS
# RESULTS SHOW OVER FITTING AS THE TRAIN ACCURACY WAS GOOD LEARNT I.E 100%
# TEST ACCURACY WAS BAD AS  THE CHANGED DATASET RESULTED IN LESS ACCURACY

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[ 58   0   0   0]
 [  0  63   0   0]
 [  0   0 128   0]
 [  0   0   0 172]]
ACCURACY SCORE:
1.0000
CLASSIFICATION REPORT:
              0     1      2      3  accuracy  macro avg  weighted avg
precision   1.0   1.0    1.0    1.0       1.0        1.0           1.0
recall      1.0   1.0    1.0    1.0       1.0        1.0           1.0
f1-score    1.0   1.0    1.0    1.0       1.0        1.0           1.0
support    58.0  63.0  128.0  172.0       1.0      421.0         421.0
TESTING RESULTS: 
CONFUSION MATRIX:
[[ 6  6  5  3]
 [ 2  6  6  7]
 [ 0  6 18  5]
 [ 3  4  0 29]]
ACCURACY SCORE:
0.5566
CLASSIFICATION REPORT:
                   0          1         2          3  accuracy   macro avg  \
precision   0.545455   0.272727   0.62069   0.659091  0.556604    0.524491   
recall      0.300000   0.285714   0.62069   0.805556  0.556604    0.502990   
f1-score    0.387097   0.279070   0.62069   0.725000  0.556604    0.502964   
support    20.000000  21.000000

In [43]:
from sklearn.ensemble import ExtraTreesClassifier

ex_tree_clf = ExtraTreesClassifier(n_estimators=1000, max_features=7, random_state=42)
ex_tree_clf.fit(X_train, Y_train)
evaluate(ex_tree_clf, X_train, X_test, Y_train, Y_test)
# SAME RESULTS
# RESULTS SHOW OVER FITTING AS THE TRAIN ACCURACY WAS GOOD LEARNT I.E 100%
# TEST ACCURACY WAS BAD AS  THE CHANGED DATASET RESULTED IN LESS ACCURACY

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[ 58   0   0   0]
 [  0  63   0   0]
 [  0   0 128   0]
 [  0   0   0 172]]
ACCURACY SCORE:
1.0000
CLASSIFICATION REPORT:
              0     1      2      3  accuracy  macro avg  weighted avg
precision   1.0   1.0    1.0    1.0       1.0        1.0           1.0
recall      1.0   1.0    1.0    1.0       1.0        1.0           1.0
f1-score    1.0   1.0    1.0    1.0       1.0        1.0           1.0
support    58.0  63.0  128.0  172.0       1.0      421.0         421.0
TESTING RESULTS: 
CONFUSION MATRIX:
[[ 4  3  9  4]
 [ 3  5  9  4]
 [ 1  3 18  7]
 [ 2  3  2 29]]
ACCURACY SCORE:
0.5283
CLASSIFICATION REPORT:
                   0          1          2          3  accuracy   macro avg  \
precision   0.400000   0.357143   0.473684   0.659091  0.528302    0.472479   
recall      0.200000   0.238095   0.620690   0.805556  0.528302    0.466085   
f1-score    0.266667   0.285714   0.537313   0.725000  0.528302    0.453674   
support    20.000000  21.00

In [44]:
from sklearn.ensemble import AdaBoostClassifier

ada_boost_clf = AdaBoostClassifier(n_estimators=30)
ada_boost_clf.fit(X_train, Y_train)
evaluate(ada_boost_clf, X_train, X_test, Y_train, Y_test)

# SAMPLE INITAILLY SHOWED UNDER FITTING  WHICH MEANS HIGH BIAS
# THE RESUKT FOR TRAIN AND TEST ACCURACY ARE BOTH LOW
# ADABOOSTING WILL TRY TO reassign weights between weak learners

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[ 26  16   6  10]
 [ 18  21   7  17]
 [  7   3  78  40]
 [  2   4  47 119]]
ACCURACY SCORE:
0.5796
CLASSIFICATION REPORT:
                   0          1           2           3  accuracy   macro avg  \
precision   0.490566   0.477273    0.565217    0.639785  0.579572    0.543210   
recall      0.448276   0.333333    0.609375    0.691860  0.579572    0.520711   
f1-score    0.468468   0.392523    0.586466    0.664804  0.579572    0.528066   
support    58.000000  63.000000  128.000000  172.000000  0.579572  421.000000   

           weighted avg  
precision      0.572237  
recall         0.579572  
f1-score       0.573193  
support      421.000000  
TESTING RESULTS: 
CONFUSION MATRIX:
[[ 9  6  2  3]
 [ 2  6  5  8]
 [ 2  2 19  6]
 [ 3  3  6 24]]
ACCURACY SCORE:
0.5472
CLASSIFICATION REPORT:
                 0          1          2          3  accuracy   macro avg  \
precision   0.5625   0.352941   0.593750   0.585366   0.54717    0.523639   
recall  

In [45]:
from sklearn.ensemble import GradientBoostingClassifier

grad_boost_clf = GradientBoostingClassifier(n_estimators=100, random_state=42)
grad_boost_clf.fit(X_train, Y_train)
evaluate(grad_boost_clf, X_train, X_test, Y_train, Y_test)

# SAMPLE INITAILLY SHOWED UNDER FITTING  WHICH MEANS HIGH BIAS
# THE RESUKT FOR TRAIN AND TEST ACCURACY ARE BOTH LOW
# stochastic classifier WILL TRY TO correct the loss between models

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[ 57   0   0   1]
 [  0  62   0   1]
 [  0   0 124   4]
 [  0   0   0 172]]
ACCURACY SCORE:
0.9857
CLASSIFICATION REPORT:
                   0          1           2           3  accuracy   macro avg  \
precision   1.000000   1.000000    1.000000    0.966292  0.985748    0.991573   
recall      0.982759   0.984127    0.968750    1.000000  0.985748    0.983909   
f1-score    0.991304   0.992000    0.984127    0.982857  0.985748    0.987572   
support    58.000000  63.000000  128.000000  172.000000  0.985748  421.000000   

           weighted avg  
precision      0.986229  
recall         0.985748  
f1-score       0.985775  
support      421.000000  
TESTING RESULTS: 
CONFUSION MATRIX:
[[ 7  2  8  3]
 [ 1  7  5  8]
 [ 0  3 17  9]
 [ 2  7  2 25]]
ACCURACY SCORE:
0.5283
CLASSIFICATION REPORT:
                   0          1          2          3  accuracy   macro avg  \
precision   0.700000   0.368421   0.531250   0.555556  0.528302    0.538807   
reca

In [46]:
# VOTING CLASSIFIER
# need to create a workign statement to prove the ML algo

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

estimators = []
log_reg = LogisticRegression(solver='liblinear')
estimators.append(('Logistic', log_reg))

tree = DecisionTreeClassifier()
estimators.append(('Tree', tree))

svm_clf = SVC(gamma='scale')
estimators.append(('SVM', svm_clf))

voting = VotingClassifier(estimators=estimators)
voting.fit(X_train, Y_train)

evaluate(voting, X_train, X_test, Y_train, Y_test)

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[ 45   0   0  13]
 [ 12  34   0  17]
 [  5   4  71  48]
 [  0   0   0 172]]
ACCURACY SCORE:
0.7648
CLASSIFICATION REPORT:
                   0          1           2           3  accuracy   macro avg  \
precision   0.725806   0.894737    1.000000    0.688000  0.764846    0.827136   
recall      0.775862   0.539683    0.554688    1.000000  0.764846    0.717558   
f1-score    0.750000   0.673267    0.713568    0.815166  0.764846    0.738000   
support    58.000000  63.000000  128.000000  172.000000  0.764846  421.000000   

           weighted avg  
precision      0.819005  
recall         0.764846  
f1-score       0.754064  
support      421.000000  
TESTING RESULTS: 
CONFUSION MATRIX:
[[ 9  2  3  6]
 [ 5  6  2  8]
 [ 4  2 15  8]
 [ 1  5  0 30]]
ACCURACY SCORE:
0.5660
CLASSIFICATION REPORT:
                   0          1          2          3  accuracy   macro avg  \
precision   0.473684   0.400000   0.750000   0.576923  0.566038    0.550152   
reca

In [46]:
# DEFINING ALL THE METRICS FOR BUSINESS VIEW
#-------------------------METRCIS = ACCURACY
ACCURACY = ( NO OF CORRECT PREDICTIONS /  TOTAL NO OF PREDICTIONS )
ACCURACY METRICS WORKS BEST ON THE BALANCED DATASET,  BUT IF THE DATASET IS IMBALANCED THEN IT DOESNT PERFORM WELL


#-------------------------METRCIS = CONFUSION MATRIX

PREDICTS THE TRUE POSITIVES, TRUE NEGATIVES, FALSE POSITIVES, FALSE NEGATIVES

TRUE POSITIVES : WHEN THE MODEL PREDICTS THE RESULT AS TRUE AND THE RESULT IS ACTUALLY TRUE
TRUE NEGATIVES : WHEN THE MODEL PREDICTS THE RESULT AS FALSE AND THE RESULT IS ACTUALLY FALSE
FALSE POSITIVES : WHEN THE MODEL PREDICTS THE RESULT AS TRUE AND THE RESULT IS ACTUALLY FALSE
FALSE NEGATIVES :  WHEN THE MODEL PREDICTS THE RESUT AS FALSE AND THE RESULT IS ACTUALLY TRUE

#------------------------------ METRICS = PRECISION 

THE MODEL PREDICTS THE RESULTS AS ALL THE PREDICTED POSITIVES = TP / (TP + FP)



#------------------------------ METRICS = RECALL also Termed as TPR(Sensitivity)

THE MODEL PREDICTS THE RESULTS AS ALL THE ACTUAL PREDICTED POSITIVES = TP / (TP + FN)



#------------------------------ METRICS = F1 SCORE 




In [46]:
# DEFINING ALL THE METRICS FOR BUSINESS VIEW FOR REGRESSION TECHNIQUES

REgression Evaluation


Residual error  =  diff between actual and predicted error

MAE =  sum(acutal -  predicted)/n

MSE  =  sum(actual - predicted)**2/n

RMSE = squareroot(sum(actual - predicted)**2/n)

RMSLE = squareroot(sum(log((actual + 1) - (predicted+1)))**2/n)

